<a href="https://colab.research.google.com/github/WanyaKruk/DZ-3/blob/main/DB_Drivers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3
import pandas as pd
data = pd.read_csv('data_encoded.csv')
data

,year,sex,region,quantity
0,2008,Оба пола,Республика Беларусь,237356.0
1,2008,Оба пола,Брестская область,51979.0
2,2008,Оба пола,Витебская область,42909.0
3,2008,Оба пола,Гомельская область,68010.0
4,2008,Оба пола,Гродненская область,13608.0
...,...,...,...,...
379,2023,Мужчины,Гомельская область,NaN
380,2023,Мужчины,Гродненская область,NaN
381,2023,Мужчины,г. Минск,NaN
382,2023,Мужчины,Минская область,NaN


In [2]:
conn = sqlite3.connect('drivers.db')
cursor = conn.cursor()

In [3]:
cursor.execute('''CREATE TABLE IF NOT EXISTS drivers(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    year INTEGER,
    sex TEXT UNIQUE NOT NULL,
    region TEXT UNIQUE NOT NULL,
    quantity INTEGER
)''')

In [4]:
cursor.execute("INSERT INTO drivers (year, sex, region, quantity) VALUES (?, ?, ?, ?)", data,)

ProgrammingError: Incorrect number of bindings supplied. The current statement uses 4, and there are 384 supplied.

In [5]:
cursor.execute('''
INSERT INTO drivers (year, sex, region, quantity)
VALUES ('2024', 'оба пола', 'Минск','3')
''')

In [6]:
cursor.execute("SELECT * FROM drivers")
drivers = cursor.fetchall()
for driver in drivers:
    print(driver)

(1, 2024, 'оба пола', 'Минск', 3)
